<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/AdiabaticFlash_Broyden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [2]:
import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True)
from scipy.optimize import root, minimize, NonlinearConstraint
import tools.che as che
R=8.314 # J/(mol K)
eps=1e-12

In [3]:
p = che.Props(['Ethanol','Isopropanol', 'Water'])

In [10]:
Ftot=10 # Total Feed moles
Fz = jnp.array([1/3, 1/3, 1/3]) # Equimolar feed composition
FT = 450 # Feed temperature
flashP= 101325 # Flash drum pressure

Vy = Fz # Guess vapor/liquid composition equal to feed
Lx = Fz # Comp - constrains mole fractions to behave like mole fractions!
flashT = jnp.array([360.])  # Guess and bounds for flash temperature
Vtot = jnp.array([Ftot/2])  # Guess half of feed in vapor
Ltot = jnp.array([Ftot/2])

x_guess = jnp.concatenate([Vy, Lx, flashT, Vtot, Ltot])

In [16]:
def model(x):
  Vy = x[0:3]
  Lx = x[3:6]
  flashT = x[6]
  Vtot = x[7]
  Ltot = x[8]

  sum_frac = jnp.array([jnp.sum(Vy)-1., jnp.sum(Lx)-1.])

  V = Vy * Vtot
  L = Lx * Ltot
  F = Fz * Ftot
  mass_balance = F-V-L

  FH = p.Hl(nL=F, T=FT)
  VH = p.Hv(nV=V, T=flashT)
  LH = p.Hl(nL=L, T=flashT)
  energy_balance = jnp.array([FH - VH - LH])

  fugL = Lx  * p.NRTL_gamma(Lx,flashT)* p.Pvap(flashT)
  fugV = Vy*flashP
  VLE = fugL - fugV
  return jnp.concatenate([sum_frac, mass_balance, energy_balance, VLE])

In [17]:
model(x_guess)

DeviceArray([     0.        ,      0.        ,      0.        ,
                  0.        ,      0.        , -48834.08861275,
              17774.2642706 ,  11514.40996588,   3103.78604499],            dtype=float64)

In [25]:
def broyden3(func, x, J=None, max_iter=100, tol=1e-6, verbose=0, xmax=jnp.inf, xmin=-jnp.inf):
  Jf = jax.jacobian(func) if J is None else J
  J = Jf(x)
  Jinv = jnp.linalg.inv(J)
  f = func(x)

  for i in range(max_iter):
    print(f"\nIter: {i}")
    dx = - Jinv @ f

    alpha_max_limits = jnp.min(jnp.where(x + dx > xmax, (xmax - x) / (dx), 1))
    alpha_min_limits = jnp.min(jnp.where(x + dx < xmin, (xmin - x) / (dx), 1))
    alpha = min(alpha_max_limits, alpha_min_limits)

    while alpha > 0.01:
      dx_try = alpha*dx
      xp = x + dx_try
      fp = func(xp)
      dnorm = jnp.linalg.norm(fp)-jnp.linalg.norm(f)
      if verbose>1:
        print(f"Alpha {alpha}   dnorm {dnorm}  dx_try {dx_try}   f {f}    fp {fp}")
      if dnorm > 0:
        alpha *= 0.5
      else:
        break
    if alpha <= 0.01:
      if verbose>1:
        print("reevaluate J")
      Jinv = jnp.linalg.inv(Jf(x))
      continue

    dx=dx_try
    f= fp
    x= xp
    if verbose>0:
      print(i, x, f)
      print()
    if jnp.linalg.norm(fp) < tol:
      break

    u = jnp.expand_dims(fp,1)
    v = jnp.expand_dims(dx,1)/jnp.linalg.norm(dx)**2
    Jinv = Jinv - Jinv @ u @ v.T @ Jinv / (1 + v.T @ Jinv @ u)  #Sherman-Morrison
  return x, f

In [ ]:
broyden3(model, x_guess, verbose=2, max_iter=100,
         xmin=jnp.array([0., 0., 0., 0., 0., 0., 300., 0., 0.]),
         xmax=jnp.array([1., 1., 1., 1., 1., 1., FT, Ftot, Ftot]))